In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
from splinter import Browser
import tweepy
from config import consumer_key, consumer_secret, access_token, access_token_secret

In [6]:
def init_browser():
    executable_path = {"executable_path": "/Users/erincullen/Downloads/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

### NASA Mars News

In [3]:
url_nasa = "https://mars.nasa.gov/news/"

In [4]:
def scrape_mars (url):
    browser = init_browser()
    browser.visit(url)
    soup = BeautifulSoup(browser.html, "html.parser")
    news = soup.find(class_="item_list")
    headline = news.find(class_="content_title").text
    para = news.find(class_="article_teaser_body").text
    print(headline)
    print(para)
    browser.quit()

In [5]:
scrape_mars(url_nasa)

NASA Mars Mission Adds Southern California Dates
Looking for summer fun? Southern California families have their choice of the beach, movies, museums -- and even NASA's next mission to Mars.


### JPL Mars Space Images

In [6]:
url_jpl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

In [44]:
import time

In [68]:
def scrape_jpl (url):
    browser = init_browser()
    browser.visit(url)

    browser.click_link_by_partial_text('FULL IMAGE')
    time.sleep(5)
    browser.click_link_by_partial_text('more info')
    time.sleep(5)

    soup = BeautifulSoup(browser.html, "html.parser")
    results = soup.find('article')
    ext = results.find("figure", class_='lede').a['href']
    base = "https://www.jpl.nasa.gov"
    featured_image_url = base + ext
    browser.quit()

    featured_image_url

scrape_jpl(url_jpl)

### Mars Weather

In [155]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [161]:
target_user = "@marswxreport" 
results = api.user_timeline(target_user)  
weather =[]
#Remove non-weather based tweets
for r in results: 
    if 'Sol'in r['text']:
        weather.append(r['text'])
    else:
        pass

mars_weather = weather[0]
mars_weather

'Sol 2108 (2018-07-12), Sunny, high -24C/-11F, low -65C/-84F, pressure at 8.06 hPa, daylight 05:19-17:27'

### Mars Facts

In [194]:
url_facts = "https://space-facts.com/mars/"

tables = pd.read_html(url_facts)
table = tables[0]

In [195]:
table = table.rename(index=str, columns={0:"Type",1:"Data"})
table = table.set_index('Type')

In [196]:
table

,Data
Type,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [10]:
browser = init_browser()
browser.visit("https://space-facts.com/mars/")
soup = BeautifulSoup(browser.html, "html.parser")
table = soup("tbody")

In [16]:
str(table[0])

'<tbody>\n<tr class="row-1 odd">\n<td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/>\n</td>\n</tr>\n<tr class="row-2 even">\n<td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/>\n</td>\n</tr>\n<tr class="row-3 odd">\n<td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.42 x 10^23 kg (10.7% Earth)</td>\n</tr>\n<tr class="row-4 even">\n<td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/deimos/">Deimos</a>)</td>\n</tr>\n<tr class="row-5 odd">\n<td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km (1.52 AU)</td>\n</tr>\n<tr class="row-6 even">\n<td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/>\n</td>\n</tr>\n<tr class="row-7 odd">\n<td class="column-1"><strong>Surface Temperature: </

### Mars Hemispheres

In [251]:
hemisphere_data = []
url_hemi = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [252]:
# image_urls = []
browser = init_browser()
browser.visit(url_hemi)
soup = BeautifulSoup(browser.html, "html.parser")
titles = soup("h3")
for t in titles: 
    browser.click_link_by_partial_text(t.text)
    soup = BeautifulSoup(browser.html, "html.parser")
    url = soup.find(class_="downloads").a['href']
    hemisphere_data.append({"title":t.text,"image_url":url})
    browser.visit(url_hemi)

hemisphere_data

[{'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]